Based on **Francesco Pierfederici: Distributed Computing with Python, Chapter 3**

### Multiprocess queues
When using multiple processes, the issue that comes up is **how to exchange data 
between the workers**. 

The multiprocessing module offers a mechanism to do that 
in the form of **queues and pipes**. 

The **multiprocessing.Queue** class is modeled after the queue.Queue class with the 
additional twist that **items stored in the multiprocessing queue need to be pickable**.


In [1]:
def fib(n):
    if n <= 2:
        return 1
    elif n == 0:
        return 0
    elif n < 0:
        raise Exception('fib(n) is undefined for n < 0')
    return fib(n - 1) + fib(n - 2)

In [2]:
def worker(inq, outq): #input queue for the task (fib function and its argument), output queue for the results (integer)
    while True:
        data = inq.get()
        if data is None:
            return
        fn, arg = data
        outq.put(fn(arg))

We use a **two-queue architecture**, whereby **one queue 
holds the tasks to be performed** (in this case, the function to be called and its only 
argument), while the **other queue holds the results** (simple integers in this example).

In [3]:
import multiprocessing as mp

workernum=4
fibnum=34

# We open two multiprocessing queues:
tasks = mp.Queue()
results = mp.Queue()

for i in range(workernum):
    tasks.put((fib, fibnum))

In [4]:
for i in range(workernum):
        mp.Process(target=worker, args=(tasks, results)).start()

In [5]:
for i in range(workernum):
        print(results.get())

5702887
5702887
5702887
5702887


In [6]:
for i in range(workernum):
        tasks.put(None)

As we did previously, we use a sentinel value (None) in the task queue to signal that 
the worker processes should quit. The worker process is a simple multiprocessing.
Process instance whose target is the worker function and whose behavior is the one 
that we just described.

Another piece of technology that might be worth investigating is **Cython**, a Python-like language to create C modules that is extremely popular and actively developed. Cython has excellent support for OpenMP, a directive-based API for C, 
C++, and Fortran, that allows programmers to easily multithread their code.